<a href="https://colab.research.google.com/github/sscc37/C.Language/blob/0529-yolov8-learning/yolov8_train_segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 초기 환경 설정

In [1]:
# YOLOv8 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
# 필요한 라이브러리 임포트
import os
from ultralytics import YOLO
import shutil
from google.colab import drive
from google.colab import files

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# 파일 선택기를 통한 업로드/ 학습된 zip 파일을 다운로드 한다.
uploaded = files.upload()

Saving test.v1i.yolov8.zip to test.v1i.yolov8.zip


In [9]:
# 코랩 셀에서 ! 를 앞에 붙여서 실행
!mkdir -p /content/dataset
!unzip /content/test.v1i.yolov8.zip -d /content/dataset

Archive:  /content/test.v1i.yolov8.zip
  inflating: /content/dataset/README.dataset.txt  
  inflating: /content/dataset/README.roboflow.txt  
  inflating: /content/dataset/data.yaml  
   creating: /content/dataset/train/
   creating: /content/dataset/train/images/
 extracting: /content/dataset/train/images/121-_jpg.rf.d36e70fd2de33b8b6cc77160a7088b6f.jpg  
 extracting: /content/dataset/train/images/122-_jpg.rf.9c46815cf488d2659f9a7258abf27029.jpg  
 extracting: /content/dataset/train/images/123-_jpg.rf.a228c18f448d90fe3ef627dd4b92b7ed.jpg  
 extracting: /content/dataset/train/images/124-_jpg.rf.9ab2f9ebb0279f89ce2b5dacb8b1083f.jpg  
 extracting: /content/dataset/train/images/125-_jpg.rf.e4a04a05e6a6374afe927c8ff9045546.jpg  
 extracting: /content/dataset/train/images/126-_jpg.rf.52a46fd704e8f5eee804bc7d95104d8e.jpg  
 extracting: /content/dataset/train/images/127-_jpg.rf.46c2cbd52bea81fb4bffd502d92cbb04.jpg  
 extracting: /content/dataset/train/images/128-_jpg.rf.026b7a4bce903ffb6b692f

다음 폴더 구조를 수정합니다.


content/

├── dataset
│   ├──train
│   │   ├──images
│   │   ├──labels
│   ├──valid
│   │   ├──images
│   ├──test
│   │   ├──images
└── data.yaml





In [5]:
#test/images 폴더 생성
!mkdir -p /content/dataset/test/images
!cp /content/dataset/train/images/* /content/dataset/test/images/

cp: cannot stat '/content/dataset/train/images/*': No such file or directory


In [6]:
#valid/images 폴더 생성
!mkdir -p /content/dataset/valid/images
!cp /content/dataset/train/images/* /content/dataset/valid/images/

cp: cannot stat '/content/dataset/train/images/*': No such file or directory


In [7]:
#valid/labels 폴더 생성
!mkdir -p /content/dataset/valid/labels
!cp /content/dataset/train/labels/* /content/dataset/valid/labels/

cp: cannot stat '/content/dataset/train/labels/*': No such file or directory


#data.yaml 파일 수정

train: /content/dataset/train/images
val: /content/dataset/valid/images
test: /content/dataset/test/images

In [11]:
# 학습 시작
from ultralytics import YOLO

# Segmentation 모델로 학습
model = YOLO('yolov8l-seg.pt')  # segmentation 모델 사용

results = model.train(
    data='/content/dataset/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='yolov8_seg_custom',
    project='/content/runs/train',
    patience=0,
    save=True,
    device=0
)

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_seg_custom3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=0, perspective=0.0, plots=True, pose=12.0, 

train: Scanning /content/dataset/train/labels.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 411.8±188.5 MB/s, size: 21.9 KB)


val: Scanning /content/dataset/train/labels.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]


...


50 epochs completed in 0.075 hours.
Optimizer stripped from /content/runs/train/yolov8_seg_custom3/weights/last.pt, 92.3MB
Optimizer stripped from /content/runs/train/yolov8_seg_custom3/weights/best.pt, 92.3MB

Validating /content/runs/train/yolov8_seg_custom3/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8l-seg summary (fused): 125 layers, 45,915,743 parameters, 0 gradients, 220.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.01it/s]


                   all         15        150      0.941      0.891      0.924       0.81      0.924      0.877      0.904      0.637
           dotted-line         15         87      0.985      0.755      0.888       0.67      0.895      0.685       0.78      0.443
             left-lane         15         15      0.994          1      0.995      0.931      0.995          1      0.995      0.757
             left-line         15         15      0.974          1      0.995      0.863      0.974          1      0.995      0.694
            right-iine         14         17      0.854      0.765      0.775      0.734      0.856      0.765      0.778      0.599
            right-lane         15         16        0.9      0.938      0.969      0.854      0.901      0.938      0.969      0.696
Speed: 0.2ms preprocess, 20.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/runs/train/yolov8_seg_custom3


In [22]:
# 학습된 모델 로드(폴더에 맞도록)
model = YOLO('/content/runs/train/yolov8_seg_custom3/weights/best.pt')

# 테스트 실행
results = model.predict(
    source='/content/dataset/train/images',  # 테스트 이미지 경로
    save=True,                              # 결과 저장
    save_txt=True,                          # 텍스트 결과도 저장
    conf=0.5,                               # 신뢰도 임계값
    project='/content/runs/detect',         # 저장 프로젝트 경로
    name='test_results'                     # 폴더 이름 지정
)


image 1/15 /content/dataset/train/images/121-_jpg.rf.d36e70fd2de33b8b6cc77160a7088b6f.jpg: 640x640 4 dotted-lines, 1 left-lane, 1 left-line, 1 right-iine, 1 right-lane, 76.0ms
image 2/15 /content/dataset/train/images/122-_jpg.rf.9c46815cf488d2659f9a7258abf27029.jpg: 640x640 5 dotted-lines, 1 left-lane, 1 left-line, 1 right-iine, 2 right-lanes, 56.2ms
image 3/15 /content/dataset/train/images/123-_jpg.rf.a228c18f448d90fe3ef627dd4b92b7ed.jpg: 640x640 4 dotted-lines, 1 left-lane, 1 left-line, 1 right-iine, 1 right-lane, 56.2ms
image 4/15 /content/dataset/train/images/124-_jpg.rf.9ab2f9ebb0279f89ce2b5dacb8b1083f.jpg: 640x640 5 dotted-lines, 1 left-lane, 1 left-line, 1 right-iine, 2 right-lanes, 56.2ms
image 5/15 /content/dataset/train/images/125-_jpg.rf.e4a04a05e6a6374afe927c8ff9045546.jpg: 640x640 4 dotted-lines, 1 left-lane, 1 left-line, 1 right-iine, 1 right-lane, 49.3ms
image 6/15 /content/dataset/train/images/126-_jpg.rf.52a46fd704e8f5eee804bc7d95104d8e.jpg: 640x640 5 dotted-lines, 1 

In [23]:
import cv2

def draw_enhanced_results(image_path, results):
    """세그멘테이션 마스크를 이미지 위에 시각화"""
    # 이미지 로드
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 클래스 색상 정의
    class_names = ['dotted-line', 'left_lane', 'left_line', 'right_lane', 'right_line']
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

    for result in results:
        masks = result.masks  # 마스크 객체

        if masks is not None and masks.data is not None:
            mask_data = masks.data.cpu().numpy()  # (N, H, W)
            cls_list = result.boxes.cls.cpu().numpy().astype(int)  # 클래스 인덱스 리스트
            conf_list = result.boxes.conf.cpu().numpy()  # 신뢰도

            for i in range(len(mask_data)):
                mask = mask_data[i]
                cls_id = cls_list[i]
                conf = conf_list[i]

                # 마스크 색상 및 클래스 이름
                color = colors[cls_id % len(colors)]
                class_name = class_names[cls_id] if cls_id < len(class_names) else f'class_{cls_id}'
                label = f'{class_name}: {conf:.2f}'

                # 마스크를 컬러로 적용
                colored_mask = np.zeros_like(img_rgb, dtype=np.uint8)
                for c in range(3):
                    colored_mask[:, :, c] = mask * color[c]

                # 반투명하게 overlay
                img_rgb = cv2.addWeighted(img_rgb, 1.0, colored_mask, 0.4, 0)

                # 마스크 경계 추출하여 텍스트 위치 추정
                contours, _ = cv2.findContours((mask * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    x, y, w, h = cv2.boundingRect(contours[0])
                    # 라벨 텍스트
                    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                    cv2.rectangle(img_rgb, (x, y - text_height - 10), (x + text_width, y), color, -1)
                    cv2.putText(img_rgb, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return img_rgb


In [24]:
import matplotlib.pyplot as plt
import numpy as np

image_dir = '/content/dataset/test/images/'
test_images = sorted([img for img in os.listdir(image_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))])


for i, img_name in enumerate(test_images[:5]):  # 처음 5개만
    img_path = f'/content/dataset/test/images/{img_name}'

    # 예측 실행
    result = model.predict(img_path, conf=0.5, verbose=False)

    # 향상된 시각화
    enhanced_img = draw_enhanced_results(img_path, result)

    # 결과 표시
    plt.figure(figsize=(12, 8))
    plt.imshow(enhanced_img)
    plt.title(f'Detection Result: {img_name}')
    plt.axis('off')
    plt.show()


In [25]:
import shutil
from google.colab import files

# 1. zip 파일로 압축 (기존 zip 파일이 있다면 덮어씌움)
shutil.make_archive('/content/runs_backup', 'zip', '/content/runs')

# 2. 다운로드 링크 제공
files.download('/content/runs_backup.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# YOLO 예측 + 시각화 함수 (콜라보용)
image_dir = '/content/dataset/train/images'
test_images = sorted([img for img in os.listdir(image_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Check if test_images is empty before proceeding
if not test_images:
    print(f"Warning: No image files found in {image_dir}. Cannot create video.")
else:
    frames = []
    target_duration = 15  # 목표 동영상 길이 (초)

    # 처음 5개만 사용하되, test_images에 이미지가 5개 미만일 경우 전체 사용
    images_to_process = test_images[:5] if len(test_images) >= 5 else test_images

    for i, img_name in enumerate(images_to_process):
        img_path = os.path.join(image_dir, img_name)

        # 예측 실행
        # Wrap the prediction call in a try-except block to catch potential issues with individual images
        try:
            result = model.predict(img_path, conf=0.5, verbose=False)

            # 향상된 시각화
            enhanced_img = draw_enhanced_results(img_path, result)

            # RGB -> BGR (OpenCV용)
            bgr_img = cv2.cvtColor(np.array(enhanced_img), cv2.COLOR_RGB2BGR)

            # 모든 프레임 사이즈를 동일하게 유지
            # Use the shape of the first successful frame as the target size
            if not frames:  # If this is the first frame
                target_height, target_width, _ = bgr_img.shape
            bgr_img = cv2.resize(bgr_img, (target_width, target_height))

            frames.append(bgr_img)
        except Exception as e:
            print(f"Error processing image {img_name}: {e}")
            continue  # Skip this image and continue with the next

    # 🔄 동영상 생성 (15초 길이, 높은 FPS)
    if frames:  # Only attempt to create video if frames were added
        video_path = "/content/detection_results_15sec.mp4"

        # 목표 설정
        target_fps = 24  # 부드러운 재생을 위한 목표 FPS
        target_total_frames = target_duration * target_fps  # 15초 * 24fps = 360프레임

        original_frame_count = len(frames)
        print(f"📊 원본 프레임 수: {original_frame_count}개")

        # 프레임이 부족하면 반복하여 목표 프레임 수 맞추기
        extended_frames = []
        frames_per_original = max(1, target_total_frames // original_frame_count)  # 각 프레임 반복 횟수

        for i in range(target_total_frames):
            frame_index = i % original_frame_count  # 순환 반복
            extended_frames.append(frames[frame_index])

        print(f"✅ 15초 동영상 생성")
        print(f"   - 목표 FPS: {target_fps}")
        print(f"   - 총 프레임: {len(extended_frames)}개")
        print(f"   - 각 이미지당 약 {target_total_frames // original_frame_count}프레임 반복")

        height, width, _ = frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 코덱 설정

        out = cv2.VideoWriter(video_path, fourcc, target_fps, (width, height))

        for frame in extended_frames:
            out.write(frame)

        out.release()
        print(f"✅ 15초 고품질 영상 저장 완료: {video_path}")

        # Colab에서 다운로드 링크 제공
        from google.colab import files
        files.download(video_path)
    else:
        print("No frames were successfully processed to create a video.")

# 🎯 추가 옵션: 프레임 반복으로 정확히 15초 만들기
def create_exact_15sec_video(frames, output_path="/content/detection_exact_15sec.mp4"):
    """프레임을 반복하여 정확히 15초 동영상 생성"""
    if not frames:
        print("프레임이 없습니다.")
        return

    target_duration = 15  # 목표 시간 (초)
    target_fps = 30       # 목표 FPS
    target_total_frames = target_duration * target_fps  # 15초 * 30fps = 450프레임

    original_frame_count = len(frames)

    # 프레임 반복하여 목표 프레임 수 맞추기
    extended_frames = []
    for i in range(target_total_frames):
        frame_index = i % original_frame_count  # 순환 반복
        extended_frames.append(frames[frame_index])

    # 동영상 생성
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    out = cv2.VideoWriter(output_path, fourcc, target_fps, (width, height))

    for frame in extended_frames:
        out.write(frame)

    out.release()

    print(f"✅ 정확히 15초 동영상 생성 완료!")
    print(f"   - 파일: {output_path}")
    print(f"   - 원본 프레임: {original_frame_count}개")
    print(f"   - 최종 프레임: {target_total_frames}개")
    print(f"   - FPS: {target_fps}")
    print(f"   - 길이: {target_duration}초")

    # Colab에서 다운로드
    from google.colab import files
    files.download(output_path)

# 정확히 15초 동영상도 생성하고 싶다면 아래 줄의 주석을 해제하세요
# if frames:
#     create_exact_15sec_video(frames)

Results saved to /content/runs/detect/test_results
15 labels saved to /content/runs/detect/test_results/labels
Results saved to /content/runs/detect/test_results
15 labels saved to /content/runs/detect/test_results/labels
Results saved to /content/runs/detect/test_results
15 labels saved to /content/runs/detect/test_results/labels
Results saved to /content/runs/detect/test_results
15 labels saved to /content/runs/detect/test_results/labels
Results saved to /content/runs/detect/test_results
15 labels saved to /content/runs/detect/test_results/labels
📊 원본 프레임 수: 5개
✅ 15초 동영상 생성
   - 목표 FPS: 24
   - 총 프레임: 360개
   - 각 이미지당 약 72프레임 반복
✅ 15초 고품질 영상 저장 완료: /content/detection_results_15sec.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>